In [ ]:
from  langchain_community.document_loaders import PyPDFLoader

In [1]:
#load data
file_path='Data/mindset.pdf'


In [7]:
loader=PyPDFLoader(file_path)
documents=loader.load()

In [8]:
len(documents)

306

In [17]:
documents[5].page_content[:500]

'CONTENTS\nIntroduction\n1.    THE MINDSETS\nWhy Do People Differ?\nWhat Does All This Mean for You? The Two Mindsets\nA View from the Two Mindsets\nSo, What’s New?\nSelf-Insight: Who Has Accurate Views of Their Assets and\nLimitations?\nWhat’s in Store\n2.    INSIDE THE MINDSETS\nIs Success About Learning—Or Proving You’re Smart?\nMindsets Change the Meaning of Failure\nMindsets Change the Meaning of Effort\nQuestions and Answers\n3.    THE TRUTH ABOUT ABILITY AND ACCOMPLISHMENT\nMindset and School Achievement\n'

## WHat is chunking
- Chunking is about 
- - designing the right chunks that retain meaning and context, 
- - Handle multiple modality(text,table,image)
- - Fit within model constraints

## Types of Chunking

### 1.Lenght based chunking

#### 1.1 Character text splitter
- First split the documents based on separator then try to get the chunk size cloer to chunk_size
- First chunking happens based on separator

In [24]:
# print(documents[9].page_content)

In [26]:
sample_text=documents[9].page_content

In [27]:
from langchain_text_splitters import CharacterTextSplitter

In [39]:
len(sample_text)

2339

In [51]:
splitter=CharacterTextSplitter(chunk_size=10,chunk_overlap=0,separator='',length_function=lambda x:x.count(" "))
chunks=splitter.split_text(sample_text)
# Created a chunk of size 70, which is longer than the specified 10

In [52]:
type(chunks)

list

In [53]:
len(chunks)

37

In [54]:
print(chunks[0])

At the end of each chapter and throughout the last chapter,


In [48]:
chunks_doc=splitter.split_documents([documents[9]])

In [49]:
len(chunks_doc)

234

In [50]:
chunks_doc[0]

Document(metadata={'producer': 'calibre (4.13.0) [https://calibre-ebook.com]', 'creator': 'calibre (4.13.0) [https://calibre-ebook.com]', 'creationdate': '2020-03-29T20:43:51+00:00', 'author': 'Carol Dweck', 'moddate': '2020-03-29T23:43:59+03:00', 'title': 'Mindset - Updated Edition: Changing The Way You think To Fulfil Your Potential', 'source': 'Data/mindset.pdf', 'total_pages': 306, 'page': 9, 'page_label': '10'}, page_content='At the end')

In [56]:
def chunking_character_text_split(
    text: str = None,
    documents: list = None,
    **kwargs
) -> list:
    """
    Split text or documents into chunks using LangChain's CharacterTextSplitter.
    
    This function provides a flexible interface for chunking either raw text strings
    or LangChain Document objects using character-based splitting with a specified
    separator.
    
    Args:
        text (str, optional): Raw text string to be split into chunks.
            If provided, the function returns a list of text strings.
        documents (list, optional): List of LangChain Document objects to be split.
            If provided, the function returns a list of Document objects with metadata.
        **kwargs: Additional keyword arguments passed to CharacterTextSplitter.
            Common parameters include:
                - chunk_size (int): Maximum size of chunks in characters. Default: 4000
                - chunk_overlap (int): Number of overlapping characters between chunks. Default: 200
                - separator (str): Character(s) to split on. Default: '\n\n'
                - length_function (callable): Function to measure text length. Default: len
                - is_separator_regex (bool): Whether separator is a regex pattern. Default: False
                - keep_separator (bool): Whether to keep the separator in chunks. Default: False
    
    Returns:
        list: A list of text chunks (if text provided) or Document objects (if documents provided).
    
    Raises:
        ValueError: If neither text nor documents are provided, or if both are provided.
        TypeError: If text is not a string or documents is not a list.
    
    Examples:
        >>> # Split raw text
        >>> text = "Line 1\\nLine 2\\nLine 3"
        >>> chunks = chunking_character_text_split(
        ...     text=text,
        ...     chunk_size=10,
        ...     chunk_overlap=2,
        ...     separator='\\n'
        ... )
        
        >>> # Split documents
        >>> from langchain.schema import Document
        >>> docs = [Document(page_content="Some text", metadata={"source": "file.txt"})]
        >>> chunks = chunking_character_text_split(
        ...     documents=docs,
        ...     chunk_size=100,
        ...     chunk_overlap=20
        ... )
    
    Notes:
        - CharacterTextSplitter splits by separator first, then tries to combine splits
          to approach chunk_size. It will NOT split within separator-delimited sections.
        - If a section between separators exceeds chunk_size, it will be kept as-is
          (resulting in a chunk larger than specified).
        - Consider using RecursiveCharacterTextSplitter for more reliable chunking
          that respects chunk_size limits.
    """
    from langchain_text_splitters import CharacterTextSplitter
    
    # Validation: Ensure exactly one input is provided
    if text is None and documents is None:
        raise ValueError("Either 'text' or 'documents' must be provided, but both are None.")
    
    if text is not None and documents is not None:
        raise ValueError("Provide either 'text' or 'documents', not both.")
    
    # Type validation
    if text is not None and not isinstance(text, str):
        raise TypeError(f"'text' must be a string, got {type(text).__name__}")
    
    if documents is not None and not isinstance(documents, list):
        raise TypeError(f"'documents' must be a list, got {type(documents).__name__}")
    
    # Initialize splitter
    splitter = CharacterTextSplitter(**kwargs)
    
    # Process based on input type
    if text:
        chunks = splitter.split_text(text)
    else:  # documents is not None
        chunks = splitter.split_documents(documents)
    
    return chunks

In [58]:
chunks=chunking_character_text_split(text=sample_text,chunk_size=100,separator='',chunk_overlap=20)

In [60]:
chunks[0]

'At the end of each chapter and throughout the last chapter, I show you\nways to apply the lessons—way'

#### 1.2 Token Text splitter

In [ ]:



# Alternative implementation using TokenTextSplitter directly
def chunking_token_text_split_alternative(
    text: str = None,
    documents: list = None,
    encoding_name: str = "cl100k_base",
    **kwargs
) -> list:
    """
    Alternative implementation using TokenTextSplitter directly.
    
    This version uses the dedicated TokenTextSplitter class instead of
    CharacterTextSplitter.from_tiktoken_encoder(). Both approaches work,
    but this gives you more direct control over the tokenization process.
    
    Parameters and usage are identical to chunking_token_text_split().
    """
    try:
        from langchain_text_splitters import TokenTextSplitter
    except ImportError:
        raise ImportError(
            "langchain_text_splitters not found. "
            "Install with: pip install langchain-text-splitters"
        )
    
    # Validation
    if text is None and documents is None:
        raise ValueError("Either 'text' or 'documents' must be provided.")
    
    if text is not None and documents is not None:
        raise ValueError("Provide either 'text' or 'documents', not both.")
    
    if text is not None and not isinstance(text, str):
        raise TypeError(f"'text' must be a string, got {type(text).__name__}")
    
    if documents is not None and not isinstance(documents, list):
        raise TypeError(f"'documents' must be a list, got {type(documents).__name__}")
    
    # Initialize TokenTextSplitter
    splitter = TokenTextSplitter(
        encoding_name=encoding_name,
        **kwargs
    )
    
    # Process
    if text:
        chunks = splitter.split_text(text)
    else:
        chunks = splitter.split_documents(documents)
    
    return chunks


sample_text = """
Artificial intelligence (AI) is transforming the world. Machine learning models
can now process vast amounts of data and make predictions with remarkable accuracy.
Natural language processing has enabled computers to understand and generate human
language, opening up new possibilities for human-computer interaction.
"""

print("=== Token-Based Splitting ===")
token_chunks = chunking_token_text_split_alternative(
    text=sample_text,
    chunk_size=50,  # 50 tokens
    chunk_overlap=10,  # 10 token overlap
    encoding_name="cl100k_base"
)

print(f"Number of chunks: {len(token_chunks)}")
for i, chunk in enumerate(token_chunks, 1):
    print(f"\nChunk {i}:")
    print(chunk)
    print(f"Length: {len(chunk)} characters")

# Compare with character-based splitting
print("\n\n=== Character-Based Splitting (for comparison) ===")
from langchain_text_splitters import CharacterTextSplitter
char_splitter = CharacterTextSplitter(
    chunk_size=200,  # 200 characters
    chunk_overlap=40,
    separator=" "
)
char_chunks = char_splitter.split_text(sample_text)

print(f"Number of chunks: {len(char_chunks)}")
for i, chunk in enumerate(char_chunks, 1):
    print(f"\nChunk {i}:")
    print(chunk)
    print(f"Length: {len(chunk)} characters")

=== Token-Based Splitting ===


ImportError: Could not import tiktoken python package. This is needed in order to for TokenTextSplitter. Please install it with `pip install tiktoken`.

## 2. Text Structure based chunking

#### 2.1 Recursive Character Text Splitter

In [68]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [69]:
sample_text = """
LangGraph is a powerful library for building stateful, multi-actor applications with LLMs. 

It excels at creating complex agentic workflows that require multiple steps and decision points. Unlike simple chains, LangGraph maintains conversation history and state across nodes.

Key features include conditional edges, tool calling, human-in-the-loop approval, and streaming support. Developers use it for NLQ-to-SQL agents, customer support bots, and data analysis pipelines.

The library integrates seamlessly with LangChain tools, OpenAI models, and vector stores. Production deployments use FastAPI endpoints that invoke compiled graphs with custom AgentState schemas.
"""

In [74]:
splitter=RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=40)
chunks=splitter.split_text(sample_text)

In [75]:
for chunk in chunks:
    print('*'*100)
    print(chunk)

****************************************************************************************************
LangGraph is a powerful library for building stateful, multi-actor applications with LLMs.
****************************************************************************************************
It excels at creating complex agentic workflows that require multiple steps and decision points. Unlike simple chains, LangGraph maintains conversation history and state across nodes.
****************************************************************************************************
Key features include conditional edges, tool calling, human-in-the-loop approval, and streaming support. Developers use it for NLQ-to-SQL agents, customer support bots, and data analysis pipelines.
****************************************************************************************************
The library integrates seamlessly with LangChain tools, OpenAI models, and vector stores. Production deployments use FastAP

#### 2.2 MarkdownHeaderTextSplitter

In [76]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [77]:
sample_text='''
# Product Manual

## Installation

1. Download the installer
2. Run `python setup.py install`
3. Verify: `pip show mypackage`

## Usage Examples

### Basic Query
```python
result = query("SELECT * FROM users")
print(result)

'''



In [97]:
splitter=MarkdownHeaderTextSplitter(headers_to_split_on=[("#","H1"),("##","H2"),("###","H3")])
chunks=splitter.split_text(sample_text)

In [98]:
for i in chunks:
    print('*'*100)
    print(i.page_content)
    print(i.metadata)

****************************************************************************************************
1. Download the installer
2. Run `python setup.py install`
3. Verify: `pip show mypackage`
{'H1': 'Product Manual', 'H2': 'Installation'}
****************************************************************************************************
```python
result = query("SELECT * FROM users")
print(result)


{'H1': 'Product Manual', 'H2': 'Usage Examples', 'H3': 'Basic Query'}


#### 2.3 HTML Header Text Splitter

In [1]:
from langchain_text_splitters import HTMLHeaderTextSplitter

In [2]:
sample_text=''' 
<!DOCTYPE html>
<html>
<body>
    <h1>Main Document</h1>
    <p>Introduction paragraph</p>
    
    <h2>Chapter 1</h2>
    <p>Chapter 1 content</p>
    
    <h3>Section 1.1</h3>
    <p>Section content</p>
    
    <h2>Chapter 2</h2>
    <p>Chapter 2 content</p>
</body>
</html>

'''

In [3]:
headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

html_splitter = HTMLHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)

chunks = html_splitter.split_text(sample_text)

In [5]:
for i in chunks:
    print('*'*100)
    print(i)

****************************************************************************************************
page_content='Main Document' metadata={'Header 1': 'Main Document'}
****************************************************************************************************
page_content='Introduction paragraph' metadata={'Header 1': 'Main Document'}
****************************************************************************************************
page_content='Chapter 1' metadata={'Header 1': 'Main Document', 'Header 2': 'Chapter 1'}
****************************************************************************************************
page_content='Chapter 1 content' metadata={'Header 1': 'Main Document', 'Header 2': 'Chapter 1'}
****************************************************************************************************
page_content='Section 1.1' metadata={'Header 1': 'Main Document', 'Header 2': 'Chapter 1', 'Header 3': 'Section 1.1'}
**********************************************

### COde specifit splitter

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, Language

In [8]:
python_code = """
class MyClass:
    def __init__(self):
        self.value = 0
    
    def method1(self):
        return self.value
    
    def method2(self):
        self.value += 1

def standalone_function():
    return "Hello"
"""

In [9]:
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON,
    chunk_size=100,
    chunk_overlap=0
)

In [10]:
chunks = python_splitter.split_text(python_code)

In [11]:
for chunk in chunks:
    print('*'*100)
    print(chunk)    

****************************************************************************************************
class MyClass:
    def __init__(self):
        self.value = 0
****************************************************************************************************
def method1(self):
        return self.value

    def method2(self):
        self.value += 1
****************************************************************************************************
def standalone_function():
    return "Hello"


### Semantic Based Splitter

In [2]:
from langchain_text_splitters import SemanticChunker

ImportError: cannot import name 'SemanticChunker' from 'langchain_text_splitters' (d:\Learning\RAG\RAG_ENV\Lib\site-packages\langchain_text_splitters\__init__.py)

In [13]:
from langchain_text_splitters import SemanticChunker
from langchain_openai import OpenAIEmbeddings

text = """
The dog ran in the park. It was very happy. The weather was sunny.
Machine learning is a subset of AI. Neural networks are powerful.
The cat slept on the couch. It purred softly.
"""



ImportError: cannot import name 'SemanticChunker' from 'langchain_text_splitters' (d:\Learning\RAG\RAG_ENV\Lib\site-packages\langchain_text_splitters\__init__.py)

In [ ]:
semantic_splitter = SemanticChunker(
    embeddings=OpenAIEmbeddings(),
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=95
)

chunks = semantic_splitter.split_text(text)
# Groups semantically similar sentences together

### Chunk evaluation

Use[ chuunkviz ](https://chunkviz.up.railway.app/]) to visualize the quality of chunking